In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_date, month, year, sum as _sum, avg, stddev, collect_list
)

# 1. Create Spark session

In [2]:
spark = SparkSession.builder.appName("ExpenseAnalysis").getOrCreate()

# 2. Load CSV

In [3]:
df = spark.read.option("header", True).option("inferSchema", True).csv("/content/cleaned_expenses (1).csv")
print("=== Original Data ===")
df.show()

=== Original Data ===
+----------+------+----------+------+-----------+--------------------+-------------+-------------------+
|expense_id|UserID|CategoryID|Amount|ExpenseDate|         Description|     Category|              Month|
+----------+------+----------+------+-----------+--------------------+-------------+-------------------+
|         1|     1|         1|  1200| 01-07-2025|Monthly grocery s...|    Groceries|2025-07-01 00:00:00|
|         2|     1|         2|   300| 02-07-2025|            Bus pass|    Transport|2025-07-01 00:00:00|
|         3|     1|         3|   500| 10-07-2025|         Movie night|Entertainment|2025-07-01 00:00:00|
|         4|     2|         1|   900| 03-07-2025|             Grocery|    Groceries|2025-07-01 00:00:00|
|         5|     2|         4|  1100| 05-07-2025|    Electricity bill|    Utilities|2025-07-01 00:00:00|
|         7|     3|         2|  4490| 12-07-2025|    Clothes purchase|     Shopping|2025-07-01 00:00:00|
|         8|     4|        10|  4

# 3. Convert ExpenseDate to proper date format

In [4]:
df = df.withColumn("date", to_date("ExpenseDate", "yyyy-MM-dd"))

# 4. Extract month and year

In [6]:
from pyspark.sql.functions import to_date, col, month, year, dayofmonth

# Convert string to proper DateType
df = df.withColumn("ExpenseDate", to_date(col("ExpenseDate"), "dd-MM-yyyy"))

# Extract month, date, and year
df = df.withColumn("month", month(col("ExpenseDate"))) \
       .withColumn("date", dayofmonth(col("ExpenseDate"))) \
       .withColumn("year", year(col("ExpenseDate")))

df.show()


+----------+------+----------+------+-----------+--------------------+-------------+-----+----+----+
|expense_id|UserID|CategoryID|Amount|ExpenseDate|         Description|     Category|month|date|year|
+----------+------+----------+------+-----------+--------------------+-------------+-----+----+----+
|         1|     1|         1|  1200| 2025-07-01|Monthly grocery s...|    Groceries|    7|   1|2025|
|         2|     1|         2|   300| 2025-07-02|            Bus pass|    Transport|    7|   2|2025|
|         3|     1|         3|   500| 2025-07-10|         Movie night|Entertainment|    7|  10|2025|
|         4|     2|         1|   900| 2025-07-03|             Grocery|    Groceries|    7|   3|2025|
|         5|     2|         4|  1100| 2025-07-05|    Electricity bill|    Utilities|    7|   5|2025|
|         7|     3|         2|  4490| 2025-07-12|    Clothes purchase|     Shopping|    7|  12|2025|
|         8|     4|        10|  4078| 2025-07-24|    Clothes purchase|     Shopping|    7| 

# 5. Monthly spend per user

In [7]:

monthly_spend = df.groupBy("UserID", "year", "month") \
                  .agg(_sum("Amount").alias("monthly_total"))
print("=== Monthly Spend by User ===")
monthly_spend.orderBy("UserID", "year", "month").show()

=== Monthly Spend by User ===
+------+----+-----+-------------+
|UserID|year|month|monthly_total|
+------+----+-----+-------------+
|     1|2025|    7|         6639|
|     2|2025|    7|         4149|
|     3|2025|    7|         4644|
|     4|2025|    7|        13027|
|     5|2025|    7|        14859|
+------+----+-----+-------------+



# 6. Average and std deviation per user

In [8]:

stats = df.groupBy("UserID").agg(
    avg("Amount").alias("avg_spend"),
    stddev("Amount").alias("std_spend")
)
print("=== Average & Std Dev Spend per User ===")
stats.show()

=== Average & Std Dev Spend per User ===
+------+---------+------------------+
|UserID|avg_spend|         std_spend|
+------+---------+------------------+
|     1|   1327.8|1745.8170579989187|
|     3|   2322.0|3066.0150032248703|
|     5|   1485.9|   939.61664168603|
|     4|   2605.4|1379.7265308748688|
|     2|   1383.0| 670.8703302427377|
+------+---------+------------------+



# 7. Join stats with original data

In [9]:
df_with_stats = df.join(stats, on="UserID")

# 8. Detect anomalies: amount > avg + 2*std

In [10]:
anomalies = df_with_stats.filter(col("Amount") > (col("avg_spend") + 2 * col("std_spend")))
print("=== Unusual High Expenses Detected ===")
anomalies.select("UserID", "date", "Amount", "Category", "avg_spend", "std_spend").show()

=== Unusual High Expenses Detected ===
+------+----+------+--------+---------+---------+
|UserID|date|Amount|Category|avg_spend|std_spend|
+------+----+------+--------+---------+---------+
+------+----+------+--------+---------+---------+



# 9. Total spend per user

In [11]:
user_total_spend = df.groupBy("UserID").agg(_sum("Amount").alias("total_spend"))
print("=== Total Spend by User ===")
user_total_spend.orderBy("total_spend", ascending=False).show()

=== Total Spend by User ===
+------+-----------+
|UserID|total_spend|
+------+-----------+
|     5|      14859|
|     4|      13027|
|     1|       6639|
|     3|       4644|
|     2|       4149|
+------+-----------+



# 10. Pivot: Spend by category per user

In [12]:
pivot_df = df.groupBy("UserID").pivot("Category").agg(_sum("Amount"))
print("=== Spend by Category per User ===")
pivot_df.show()

=== Spend by Category per User ===
+------+------+---------+-------------+-------+---------+----------+--------+---------+------+---------+
|UserID|Dining|Education|Entertainment|Fitness|Groceries|Healthcare|Shopping|Transport|Travel|Utilities|
+------+------+---------+-------------+-------+---------+----------+--------+---------+------+---------+
|     1|  4377|     NULL|          500|    262|     1200|      NULL|    NULL|      300|  NULL|     NULL|
|     3|  NULL|     NULL|         NULL|   NULL|     NULL|      NULL|    4490|      154|  NULL|     NULL|
|     5|  2454|     3184|         NULL|   NULL|     NULL|      3143|    4277|     NULL|   111|     1690|
|     4|  NULL|      788|         NULL|   NULL|     NULL|      NULL|    7429|     NULL|  NULL|     4810|
|     2|  NULL|     2149|         NULL|   NULL|      900|      NULL|    NULL|     NULL|  NULL|     1100|
+------+------+---------+-------------+-------+---------+----------+--------+---------+------+---------+



# 11. Average monthly spend by category

In [16]:
from pyspark.sql.functions import round, avg

avg_monthly_category = df.groupBy("year", "month", "Category") \
                         .agg(round(avg("Amount"),2).alias("avg_spend"))
print("=== Average Monthly Spend per Category ===")
avg_monthly_category.orderBy("year", "month", "Category").show()

=== Average Monthly Spend per Category ===
+----+-----+-------------+---------+
|year|month|     Category|avg_spend|
+----+-----+-------------+---------+
|2025|    7|       Dining|   3415.5|
|2025|    7|    Education|  1530.25|
|2025|    7|Entertainment|    500.0|
|2025|    7|      Fitness|    262.0|
|2025|    7|    Groceries|   1050.0|
|2025|    7|   Healthcare|   3143.0|
|2025|    7|     Shopping|  2313.71|
|2025|    7|    Transport|    227.0|
|2025|    7|       Travel|    111.0|
|2025|    7|    Utilities|   1900.0|
+----+-----+-------------+---------+



# 12. Detect missing months for each user

In [17]:
all_months = set(range(1, 13))
user_months = df.select("UserID", "month").distinct() \
                .groupBy("UserID") \
                .agg(collect_list("month").alias("months_present"))

# Calculate missing months

In [18]:
user_month_data = user_months.collect()
missing_months_data = [
    (row["UserID"], sorted(list(all_months - set(row["months_present"]))))
    for row in user_month_data
]
missing_months_df = spark.createDataFrame(missing_months_data, ["UserID", "missing_months"])

print("=== Missing Months per User ===")
missing_months_df.orderBy("UserID").show(truncate=False)


=== Missing Months per User ===
+------+------------------------------------+
|UserID|missing_months                      |
+------+------------------------------------+
|1     |[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12]|
|2     |[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12]|
|3     |[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12]|
|4     |[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12]|
|5     |[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12]|
+------+------------------------------------+

